In [ ]:
!pip install azure-storage-blob

In [ ]:
# Azure 스토리지를 관리하기 위해 필요한 패키지
import os, uuid
# uuid 임시변수
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# Blob 서비스에 연결

# 애저 포탈에서 스토리지-액세스 키에서 연결 문자열 이용
connect_string = '*************************'  

# Blob 스토리지 서비스에 연결가능한 클라이언트 생성(Blob 서비스에 연결)
blob_service_client = BlobServiceClient.from_connection_string(connect_string)

In [ ]:
# Container 생성

# container 이름 생성
container_name = str(uuid.uuid4()) # 겹치지 않는 난수의 문자열을 생성
print(container_name)

# 생성된 이름으로 container 생성
container_client = blob_service_client.create_container(container_name)

#생성된 container를 생성되었는지 애저에서 확인

d8718b34-c84c-477f-a5f7-c4dbef986e0b


In [ ]:
# 데이터를 직접 upload하는 작업

# 데이터를 업로드할 경로 생성
local_path = './data'
os.mkdir(local_path)

# 로컬 파일 생성
local_file_name = str(uuid.uuid4()) + '.txt'

# 업로드 파일의 전체 경로 잡기
upload_file_path = os.path.join(local_path, local_file_name)

In [ ]:
# 업로드 할 파일을 준비
# 테스트
file = open(file=upload_file_path, mode='w') # 파일을 'w' 쓰기모드로 연다
file.write('Hello Azure Storage')
file.close()

# 새로운 blob 클라이언트 생성을 요청(파일이 업로드될 공간을 설정)
blob_client = blob_service_client.get_blob_client(
    container = container_name,
    blob = local_file_name,    
)

# 실제로 파일 업로드하는 부분
# binary 단위 형태로 읽는 모드(이래야 이미지가 안깨짐)
with open(file = upload_file_path, mode = 'rb') as data:
  blob_client.upload_blob(data)

In [ ]:
# Blob내 파일 목록 확인
blob_list = container_client.list_blobs()
for blob in blob_list:
  print('\t' +blob.name)

	c7344cf2-2003-44b3-abdb-54c935d0e458.txt


In [ ]:
# 업로드 된 파일을 불러올수 있는지(다운로드가 가능한지)

# 로컬에 다운로드(원래 원본과 다르게)
download_file_path = os.path.join(local_path, str.replace(local_file_name, 'txt', 'DOWNLOAD.txt'))

# 새로운 container 클라이언트 생성을 요청(파일이 업로드될 공간을 설정)
container_client = blob_service_client.get_container_client(container=container_name)

# 스트림형태로 날아오는 데이터를 받아야함
with open(file=download_file_path, mode = 'wb') as download_file:
  download_file.write(container_client.download_blob(blob.name).readall())   # blob속에 있는 파일 다운로드 작업

In [ ]:
# 실습으로 만들었던 자원들 정리

print('Press the Enter key to begin clean up')
input()

# container 삭제
print('Deleting blob container...')
container_client.delete_container()

# 로컬과 다운로드 받은 파일 삭제
print('Deleting the local source and downloaded files')
os.remove(upload_file_path)
os.remove(download_file_path)
os.rmdir(local_path)  # local 디렉토리 삭제

print('Done')

Press the Enter key to begin clean up

Deleting blob container...
Deleting the local source and downloaded files


FileNotFoundError: ignored